## Notebook for making predictions with deleted constant y samples


In [119]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [120]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [121]:
X_A_train = dm.data_A.iloc[:, 2:-1]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:-1]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:-1]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

Y_A_train


0          0.00
1          0.00
2          0.00
3          0.00
5        251.02
          ...  
23219      9.02
23220      0.00
23221      0.00
23222      0.00
23223      0.00
Name: pv_measurement, Length: 16948, dtype: float64

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [122]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

In [123]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

Learning rate set to 0.06403
0:	learn: 1219.3419335	total: 38ms	remaining: 37.9s
1:	learn: 1161.8217316	total: 42.3ms	remaining: 21.1s
2:	learn: 1107.6154069	total: 47.5ms	remaining: 15.8s
3:	learn: 1058.6297508	total: 52.2ms	remaining: 13s
4:	learn: 1013.2247961	total: 56.5ms	remaining: 11.2s
5:	learn: 970.7050599	total: 62.7ms	remaining: 10.4s
6:	learn: 932.5847128	total: 67.5ms	remaining: 9.57s
7:	learn: 896.9399756	total: 80ms	remaining: 9.93s
8:	learn: 862.8320193	total: 84.9ms	remaining: 9.35s
9:	learn: 831.7959589	total: 89.4ms	remaining: 8.85s
10:	learn: 804.3865116	total: 95.1ms	remaining: 8.55s
11:	learn: 777.6324088	total: 99.9ms	remaining: 8.23s
12:	learn: 753.1043962	total: 104ms	remaining: 7.93s
13:	learn: 730.3596332	total: 109ms	remaining: 7.69s
14:	learn: 710.3440706	total: 113ms	remaining: 7.44s
15:	learn: 691.9637220	total: 117ms	remaining: 7.21s
16:	learn: 675.3732813	total: 121ms	remaining: 7.01s
17:	learn: 660.4312619	total: 126ms	remaining: 6.85s
18:	learn: 646.0

## Preds

In [124]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

In [125]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("log_sub_18.csv")

df.shape

(2160, 1)